In [1]:
%matplotlib qt

import scipy.io as sio
import os
#from EEGs_persistence import *
from preprocess_data import *
from TDApipeline import *
from intensities_pipe import *
import time  
import sklearn.pipeline as skppl
import sklearn.linear_model as skllm
import sklearn.model_selection as skms
import sklearn.metrics as skm
import matplotlib.pyplot as plt
import sklearn.preprocessing as skprp
import pandas as pd
import sklearn.neighbors as sklnn
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D
'''
from joblib import Memory
from shutil import rmtree

from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
'''


def define_subject_dir(i_sub):
    """
    Creates the directory if it doesn't exist
    :param i_sub: subject id
    :return: directory path
    """
    res_dir = "results/subject_" + str(i_sub) + "/"
    if not os.path.exists(res_dir):
        print("create directory:", res_dir)
        os.makedirs(res_dir)
    return res_dir

def load_data(i_sub,space='both'):
    """
    Loads data from electrode space, font space 
    or both for a given subject
    :param i_sub: subject id
    :param space: electrode/font_space
    :return: data,directory path
    """
    subj_dir = define_subject_dir(i_sub)
    raw_data = sio.loadmat('data/dataClean-ICA3-'+str(i_sub)+'-T1.mat')
    
    if space=='electrodeSpace':
        elec_space=raw_data['dataSorted'] # [N,T,n_trials,motiv] 
        return elec_space,subj_dir
    elif space=='fontSpace':
        font_space=raw_data['ic_data3']
        return font_space,subj_dir
    else:
        elec_space=raw_data['dataSorted'] # [N,T,n_trials,motiv] 
        font_space=raw_data['ic_data3']
        return (elec_space,font_space),subj_dir
    


if __name__ == "__main__":
    subjects=[27]
    
    intensities=False
    exploratory=False
    classification=False
    PCA=True
    last=False
    
    
    
    bands=[2]#,0,1,2]  
    #bands=[-1,2]
    n_band=len(bands)
    measures=["euclidean","correlation","quaf"]#,"dtw"]
    #measures=["quaf","dtw"]
    n_measure=len(measures)
    dimensions=["zero","one"]
    #dimensions=[]
    n_dim=len(dimensions)
    feat_vect=[DimensionLandScape(),DimensionSilhouette(),TopologicalDescriptors()]
    
    
    data_table=np.zeros((2*len(subjects),21))
    subj_t=0
    
    n_vectors=len(feat_vect)
    for subject in subjects:
        space='both'
        data_space,subj_dir=load_data(subject,space=space)

        spaces=['electrodeSpace','fontSpace']
        for sp in range(1):
            t=time.time()
            space=spaces[sp]
            if not os.path.exists(subj_dir+space):
                print("create directory(plot):",subj_dir+space)
                os.makedirs(subj_dir+'/'+space)
            print('cleaning and filtering data of',space,'of subject',subject)
            preprocessor=Preprocessor(data_space[sp])
            #filtered_ts_dic=preprocessor.get_filtered_ts_dic()
            ts_band,labels_original=preprocessor.get_trials_and_labels()
            
            data_table[subj_t,0]=preprocessor.N
            data_table[subj_t,1]=ts_band.shape[0]

            
            if intensities:
                for i_band in bands:
                    print('intensities for band ', i_band)
                    intensity(subj_dir,space,ts_band,labels,i_band)

            data_table[subj_t,6]=(labels_original==0).sum()
            data_table[subj_t,7]=(labels_original==1).sum()
            data_table[subj_t,8]=(labels_original==2).sum()
            

            vectors=[]
            sing_vals=[]
            
            blocs=[[0,6,8,9,4,5],[1,7,2,3,10,11]]
            band_dic={-1: 'noFilter', 0:'alpha',1:'betta',2:'gamma'}
            if PCA: 

                N=ts_band.shape[-1]
                persistence={}
                persistence_2d={}
                for i_band in bands:
                    pca_M0=[]
                    pca_M1=[]
                    pca_M2=[]
                    persistence[i_band]={}
                    persistence_2d[i_band]={}
                    print('global picture of band',band_dic[i_band] )
                    for bl in blocs:
                        #bloc=ts_band[:,i_band,:,:][temp]
                        PC=np.abs(ts_band[:,i_band,:,:]).mean(axis=1)
                        PC=PC.reshape((-1,N))
                        labels=labels_original
                        PC,labels=preprocessor.reject_outliers(PC,labels)
                        
                        tr2bl=preprocessor.tr2bl
                        temp=[tr_bl in bl for tr_bl in tr2bl]
                        PC=PC[temp]
                        labels=labels[temp]

                        data_table[subj_t,3+i_band]=PC.shape[0]

                        X =(PC - np.mean(PC, axis=0)).T
                        n = X.shape[1]
                        Y =  X.T/np.sqrt(n-1)
                        
                        u, s, vh = la.svd(Y, full_matrices=False)
                        r=np.sum(np.where(s>1e-12,1,0))
                        #pca = vh[:r,:] @ X[:,:] # Principal components
                        variance_prop = s[:r]**2/np.sum(s[:r]**2) # Variance captured
                        acc_variance = np.cumsum(variance_prop)
                        std = s[:r]

                        '''
                        fig, axs = plt.subplots(1, 2, figsize=(18, 4))

                        # 3/4 of the total variance rule
                        axs[0].scatter(range(len(acc_variance)),acc_variance*100)
                        axs[0].set_xticks(range(len(acc_variance)), minor=False)
                        axs[0].hlines(75, xmin=0, xmax=len(std), colors='r', linestyles='dashdot')
                        axs[0].set_title('3/4 of the total variance rule')
                        axs[0].set_xlabel('PCA coordinates')
                        axs[0].set_ylabel('accumulated variance')
                        # Kraiser rule: Keep PC with eigenvalues > 1
                        # Scree plot: keep PCs before elbow
                        axs[1].scatter(range(len(std)),(std**2))
                        axs[1].set_xticks(range(len(acc_variance)), minor=False)

                        axs[1].hlines(1, xmin=0, xmax=len(std), colors='r', linestyles='dashdot')
                        axs[1].set_title('Scree Plot')
                        axs[1].set_xlabel('PCA coordinates')
                        axs[1].set_ylabel('eigenvalue')

                        if not os.path.exists(subj_dir+space+'/PCA/'+band_dic[i_band] ):
                            print("create directory(plot):",subj_dir+space+'/PCA/'+band_dic[i_band] )
                            os.makedirs(subj_dir+space+'/PCA/'+band_dic[i_band] )
                        plt.savefig(subj_dir+space+'/PCA/'+band_dic[i_band]+'/pca_plots.png')
                        plt.close()
                        print('acumulated variance:',acc_variance)'''
                        vectors.append(vh[:3,:])
                        sing_vals.append(s[:3])
                        #pca = vh[:3,:] @ X[:,:] 
                        pca = vectors[0] @ X[:,:] 
                        pca=pca.T

                        #fig.add_subplot(projection='3d')
                        pca_M0.append(pca[labels==0])
                        pca_M1.append(pca[labels==1])
                        pca_M2.append(pca[labels==2])


                    fig = plt.figure(figsize=[18,8])
                    ax =fig.add_subplot(1, 1, 1, projection='3d')
                    fig.add_axes(ax)

                    ax.scatter(pca_M0[0][:,0],pca_M0[0][:,1],pca_M0[0][:,2],label='M0_0',c='r',alpha=0.5,zdir='z')
                    ax.scatter(pca_M1[0][:,0],pca_M1[0][:,1],pca_M1[0][:,2],label='M1_0',c='g',alpha=0.5,zdir='z')
                    ax.scatter(pca_M2[0][:,0],pca_M2[0][:,1],pca_M2[0][:,2],label='M2_0',c='b',alpha=0.5,zdir='z')
                    
                    
                    
                    angle3=np.dot(vectors[0][0],vectors[1][0])
                    angle2=np.dot(vectors[0][1],vectors[1][1])
                    angle1=np.dot(vectors[0][2],vectors[1][2])
                    
                    rotmat1=np.array([[1,0,0],[0,np.cos(angle1),-np.sin(angle1)],[0,np.sin(angle1),np.cos(angle1)]])
                    rotmat2=np.array([[np.cos(angle2),0,np.sin(angle2)],[0,1,0],[-np.sin(angle2),0,np.cos(angle2)]])
                    rotmat3=np.array([[np.cos(angle3),-np.sin(angle3),0],[np.sin(angle3),np.cos(angle3),0],[0,0,1]])
                    rotmat=rotmat3 @ rotmat2 @ rotmat1
    
                    pca_M0[1]=pca_M0[1] @ rotmat
                    pca_M1[1]=pca_M1[1] @ rotmat
                    pca_M2[1]=pca_M2[1] @ rotmat
                
                    
                
                    
                    ax.scatter(pca_M0[1][:,0],pca_M0[1][:,1],pca_M0[1][:,2],label='M0_1',c='y',alpha=0.5,zdir='z')
                    ax.scatter(pca_M1[1][:,0],pca_M1[1][:,1],pca_M1[1][:,2],label='M1_1',c='k',alpha=0.5,zdir='z')
                    ax.scatter(pca_M2[1][:,0],pca_M2[1][:,1],pca_M2[1][:,2],label='M2_1',c='m',alpha=0.5,zdir='z')                    
                    
                    ax.legend()
                    ax.set_title(band_dic[i_band]+' pca projection PC')

                    ax.set_xlim3d(-1, 1)
                    ax.set_ylim3d(-1, 1)
                    ax.set_zlim3d(-1, 1)

                    ax.set_xlabel('$X$')
                    ax.set_ylabel('$Y$')
                    ax.set_zlabel('$Z$')
                    plt.show()
                    

                                 
                    





                    

        

                    
                    
                

                
                    

cleaning and filtering data of electrodeSpace of subject 27
there are 50 clean channels
global picture of band gamma


In [2]:
np.concatenate((pca_M0[0],pca_M1[0],pca_M2[0]),axis=0).mean()

-5.582574302082188e-17

In [3]:
np.concatenate((pca_M0[1],pca_M1[1],pca_M2[1]),axis=0).mean()

-1.1605531350748302e-16

In [4]:
np.concatenate((pca_M0[0],pca_M1[0],pca_M2[0]),axis=0).std()

0.10381047460785589

In [5]:
np.concatenate((pca_M0[1],pca_M1[1],pca_M2[1]),axis=0).std()

0.23368707601785085

In [6]:
sing_vals

[array([0.14060611, 0.09082384, 0.06603667]),
 array([0.40668021, 0.18787594, 0.15927706])]

In [7]:
vectors[0].T @rotmat

array([[ 7.74716275e-02, -1.88829000e-01, -6.83604346e-02],
       [ 6.67404581e-02, -5.52016921e-01, -5.33415654e-02],
       [ 6.47467557e-01, -3.32908688e-02,  7.45019456e-01],
       [ 6.23819618e-02, -2.73694610e-01, -5.07478231e-02],
       [ 4.73542902e-02, -4.47455439e-01, -1.30048774e-01],
       [ 1.34783238e-01, -1.15750754e-01, -1.42858639e-01],
       [ 1.24071751e-01,  3.55960804e-02, -9.94042412e-02],
       [ 4.90514796e-02, -2.83838362e-01, -1.89294752e-02],
       [ 5.67645529e-02, -9.78584681e-02, -3.61435444e-02],
       [ 5.37234266e-02, -1.95273510e-01, -4.85969683e-02],
       [ 2.99237508e-01,  1.71794843e-01, -3.23987016e-01],
       [ 6.70135847e-02, -1.37377972e-01, -7.09723194e-02],
       [ 4.91062442e-02, -6.73039475e-04, -4.07368782e-02],
       [ 6.70612600e-02, -5.69562202e-02, -4.51876430e-02],
       [ 8.77697355e-02, -5.67457402e-03, -5.91280988e-02],
       [ 9.25749589e-02, -1.35426311e-01, -1.07723477e-01],
       [ 2.05396167e-01,  9.10467987e-02

In [8]:
vectors[1].T

array([[ 0.30528058, -0.20142496, -0.05513904],
       [ 0.43837776, -0.47613857, -0.19108482],
       [ 0.13355514, -0.06712118, -0.02826312],
       [ 0.22620838, -0.17539397, -0.03418518],
       [ 0.40711906, -0.22240201, -0.08258438],
       [ 0.25993951,  0.2139482 ,  0.17760738],
       [-0.03817573,  0.00818586,  0.15095889],
       [ 0.13768107,  0.03916916,  0.02015656],
       [ 0.1089774 ,  0.04143455,  0.03579154],
       [ 0.1811395 , -0.12526979, -0.05446345],
       [ 0.06131674,  0.04627181,  0.03945192],
       [ 0.24428185,  0.17111447,  0.14587292],
       [ 0.04007501,  0.0539028 ,  0.02401057],
       [ 0.09998377,  0.08962039,  0.07233005],
       [ 0.08306152,  0.0926372 ,  0.06587007],
       [ 0.12716918,  0.10270973,  0.07002366],
       [ 0.06523076,  0.14404846,  0.01551149],
       [ 0.07320981,  0.01066726,  0.01494804],
       [ 0.11345544,  0.07224176,  0.07157639],
       [ 0.20894713,  0.14860273,  0.19899653],
       [ 0.08432908,  0.08875486,  0.050

In [18]:
%matplotlib qt

import scipy.io as sio
import os
#from EEGs_persistence import *
from preprocess_data import *
from TDApipeline import *
from intensities_pipe import *
import time  
import sklearn.pipeline as skppl
import sklearn.linear_model as skllm
import sklearn.model_selection as skms
import sklearn.metrics as skm
import matplotlib.pyplot as plt
import sklearn.preprocessing as skprp
import pandas as pd
import sklearn.neighbors as sklnn
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D
'''
from joblib import Memory
from shutil import rmtree

from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
'''


def define_subject_dir(i_sub):
    """
    Creates the directory if it doesn't exist
    :param i_sub: subject id
    :return: directory path
    """
    res_dir = "results/subject_" + str(i_sub) + "/"
    if not os.path.exists(res_dir):
        print("create directory:", res_dir)
        os.makedirs(res_dir)
    return res_dir

def load_data(i_sub,space='both'):
    """
    Loads data from electrode space, font space 
    or both for a given subject
    :param i_sub: subject id
    :param space: electrode/font_space
    :return: data,directory path
    """
    subj_dir = define_subject_dir(i_sub)
    raw_data = sio.loadmat('data/dataClean-ICA3-'+str(i_sub)+'-T1.mat')
    
    if space=='electrodeSpace':
        elec_space=raw_data['dataSorted'] # [N,T,n_trials,motiv] 
        return elec_space,subj_dir
    elif space=='fontSpace':
        font_space=raw_data['ic_data3']
        return font_space,subj_dir
    else:
        elec_space=raw_data['dataSorted'] # [N,T,n_trials,motiv] 
        font_space=raw_data['ic_data3']
        return (elec_space,font_space),subj_dir
    


if __name__ == "__main__":
    subjects=[27]
    
    intensities=False
    exploratory=False
    classification=False
    PCA=True
    last=False
    
    
    
    bands=[2]#,0,1,2]  
    #bands=[-1,2]
    n_band=len(bands)
    measures=["euclidean","correlation","quaf"]#,"dtw"]
    #measures=["quaf","dtw"]
    n_measure=len(measures)
    dimensions=["zero","one"]
    #dimensions=[]
    n_dim=len(dimensions)
    feat_vect=[DimensionLandScape(),DimensionSilhouette(),TopologicalDescriptors()]
    
    
    data_table=np.zeros((2*len(subjects),21))
    subj_t=0
    
    n_vectors=len(feat_vect)
    for subject in subjects:
        space='both'
        data_space,subj_dir=load_data(subject,space=space)

        spaces=['electrodeSpace','fontSpace']
        for sp in range(1):
            t=time.time()
            space=spaces[sp]
            if not os.path.exists(subj_dir+space):
                print("create directory(plot):",subj_dir+space)
                os.makedirs(subj_dir+'/'+space)
            print('cleaning and filtering data of',space,'of subject',subject)
            preprocessor=Preprocessor(data_space[sp])
            #filtered_ts_dic=preprocessor.get_filtered_ts_dic()
            ts_band,labels_original=preprocessor.get_trials_and_labels()
            
            data_table[subj_t,0]=preprocessor.N
            data_table[subj_t,1]=ts_band.shape[0]

            
            if intensities:
                for i_band in bands:
                    print('intensities for band ', i_band)
                    intensity(subj_dir,space,ts_band,labels,i_band)

            data_table[subj_t,6]=(labels_original==0).sum()
            data_table[subj_t,7]=(labels_original==1).sum()
            data_table[subj_t,8]=(labels_original==2).sum()
            

            vectors=[]
            sing_vals=[]
            
            blocs=[[0,6,8,9,4,5],[1,7,2,3,10,11]]
            band_dic={-1: 'noFilter', 0:'alpha',1:'betta',2:'gamma'}
            if PCA: 

                N=ts_band.shape[-1]
                persistence={}
                persistence_2d={}
                for i_band in bands:
                    pca_M0=[]
                    pca_M1=[]
                    pca_M2=[]
                    persistence[i_band]={}
                    persistence_2d[i_band]={}
                    print('global picture of band',band_dic[i_band] )
                    for bl in blocs:
                        #bloc=ts_band[:,i_band,:,:][temp]
                        PC=np.abs(ts_band[:,i_band,:,:]).mean(axis=1)
                        PC=PC.reshape((-1,N))
                        labels=labels_original
                        PC,labels=preprocessor.reject_outliers(PC,labels)
                        
                        tr2bl=preprocessor.tr2bl
                        temp=[tr_bl in bl for tr_bl in tr2bl]
                        PC=PC[temp]
                        labels=labels[temp]

                        data_table[subj_t,3+i_band]=PC.shape[0]

                        X =(PC - np.mean(PC, axis=0)).T
                        n = X.shape[1]
                        Y =  X.T/np.sqrt(n-1)
                        
                        u, s, vh = la.svd(Y, full_matrices=False)
                        r=np.sum(np.where(s>1e-12,1,0))
                        #pca = vh[:r,:] @ X[:,:] # Principal components
                        variance_prop = s[:r]**2/np.sum(s[:r]**2) # Variance captured
                        acc_variance = np.cumsum(variance_prop)
                        std = s[:r]

                        '''
                        fig, axs = plt.subplots(1, 2, figsize=(18, 4))

                        # 3/4 of the total variance rule
                        axs[0].scatter(range(len(acc_variance)),acc_variance*100)
                        axs[0].set_xticks(range(len(acc_variance)), minor=False)
                        axs[0].hlines(75, xmin=0, xmax=len(std), colors='r', linestyles='dashdot')
                        axs[0].set_title('3/4 of the total variance rule')
                        axs[0].set_xlabel('PCA coordinates')
                        axs[0].set_ylabel('accumulated variance')
                        # Kraiser rule: Keep PC with eigenvalues > 1
                        # Scree plot: keep PCs before elbow
                        axs[1].scatter(range(len(std)),(std**2))
                        axs[1].set_xticks(range(len(acc_variance)), minor=False)

                        axs[1].hlines(1, xmin=0, xmax=len(std), colors='r', linestyles='dashdot')
                        axs[1].set_title('Scree Plot')
                        axs[1].set_xlabel('PCA coordinates')
                        axs[1].set_ylabel('eigenvalue')

                        if not os.path.exists(subj_dir+space+'/PCA/'+band_dic[i_band] ):
                            print("create directory(plot):",subj_dir+space+'/PCA/'+band_dic[i_band] )
                            os.makedirs(subj_dir+space+'/PCA/'+band_dic[i_band] )
                        plt.savefig(subj_dir+space+'/PCA/'+band_dic[i_band]+'/pca_plots.png')
                        plt.close()
                        print('acumulated variance:',acc_variance)'''
                        vectors.append(vh[:3,:])
                        sing_vals.append(s[:3])
                        pca = vh[:3,:] @ X[:,:] 
                        #pca = vectors[0] @ X[:,:] 
                        pca=pca.T

                        #fig.add_subplot(projection='3d')
                        pca_M0.append(pca[labels==0])
                        pca_M1.append(pca[labels==1])
                        pca_M2.append(pca[labels==2])


                    fig = plt.figure(figsize=[18,8])
                    ax =fig.add_subplot(1, 1, 1, projection='3d')
                    fig.add_axes(ax)

                    ax.scatter(pca_M0[0][:,0],pca_M0[0][:,1],pca_M0[0][:,2],label='M0_0',c='r',alpha=0.5,zdir='z')
                    ax.scatter(pca_M1[0][:,0],pca_M1[0][:,1],pca_M1[0][:,2],label='M1_0',c='g',alpha=0.5,zdir='z')
                    ax.scatter(pca_M2[0][:,0],pca_M2[0][:,1],pca_M2[0][:,2],label='M2_0',c='b',alpha=0.5,zdir='z')
                    
                    
                    
                    ax.scatter(pca_M0[1][:,0],pca_M0[1][:,1],pca_M0[1][:,2],label='M0_1',c='y',alpha=0.5,zdir='z')
                    ax.scatter(pca_M1[1][:,0],pca_M1[1][:,1],pca_M1[1][:,2],label='M1_1',c='k',alpha=0.5,zdir='z')
                    ax.scatter(pca_M2[1][:,0],pca_M2[1][:,1],pca_M2[1][:,2],label='M2_1',c='m',alpha=0.5,zdir='z')                    
                    
                    ax.legend()
                    ax.set_title(band_dic[i_band]+' pca projection PC')

                    ax.set_xlim3d(-1, 1)
                    ax.set_ylim3d(-1, 1)
                    ax.set_zlim3d(-1, 1)

                    ax.set_xlabel('$X$')
                    ax.set_ylabel('$Y$')
                    ax.set_zlabel('$Z$')
                    plt.show()
                    

                                 
                    





                    

        

                    
                    
                

                
                    

cleaning and filtering data of electrodeSpace of subject 27
there are 50 clean channels
global picture of band gamma


In [19]:
np.concatenate((pca_M0[0],pca_M1[0],pca_M2[0]),axis=0).mean()


-5.582574302082188e-17

In [20]:
np.concatenate((pca_M0[1],pca_M1[1],pca_M2[1]),axis=0).mean()

4.5474735088646414e-17

In [21]:
np.concatenate((pca_M0[0],pca_M1[0],pca_M2[0]),axis=0).std()

0.10381047460785589

In [22]:
np.concatenate((pca_M0[1],pca_M1[1],pca_M2[1]),axis=0).std()

0.27428309996383404

In [23]:
sing_vals

[array([0.14060611, 0.09082384, 0.06603667]),
 array([0.40668021, 0.18787594, 0.15927706])]

In [24]:
vectors[0].T

array([[ 0.1592782 , -0.11425959, -0.08891903],
       [ 0.40483824, -0.38060335, -0.05717853],
       [ 0.68080008,  0.61076257,  0.37263499],
       [ 0.2126578 , -0.17931029, -0.06324612],
       [ 0.29699412, -0.3414736 , -0.12067209],
       [ 0.124433  , -0.05154906, -0.18393606],
       [ 0.02732938,  0.06694188, -0.14599306],
       [ 0.2205369 , -0.18406694, -0.02848553],
       [ 0.09333278, -0.04952807, -0.05422926],
       [ 0.15402181, -0.12655809, -0.06033197],
       [-0.01868773,  0.19606711, -0.43038657],
       [ 0.11640568, -0.08384393, -0.08843365],
       [ 0.02047007,  0.01526197, -0.05847579],
       [ 0.06943851, -0.01655481, -0.06846443],
       [ 0.0438482 ,  0.02857694, -0.09215505],
       [ 0.12070868, -0.079402  , -0.13281376],
       [ 0.00216087,  0.11132831, -0.30417357],
       [ 0.07175555, -0.03353419, -0.05856611],
       [ 0.06769783, -0.04374685, -0.07640896],
       [ 0.04990695, -0.00950711, -0.0817408 ],
       [ 0.03243873,  0.01727494, -0.087

In [25]:
vectors[1].T

array([[ 0.30528058, -0.20142496, -0.05513904],
       [ 0.43837776, -0.47613857, -0.19108482],
       [ 0.13355514, -0.06712118, -0.02826312],
       [ 0.22620838, -0.17539397, -0.03418518],
       [ 0.40711906, -0.22240201, -0.08258438],
       [ 0.25993951,  0.2139482 ,  0.17760738],
       [-0.03817573,  0.00818586,  0.15095889],
       [ 0.13768107,  0.03916916,  0.02015656],
       [ 0.1089774 ,  0.04143455,  0.03579154],
       [ 0.1811395 , -0.12526979, -0.05446345],
       [ 0.06131674,  0.04627181,  0.03945192],
       [ 0.24428185,  0.17111447,  0.14587292],
       [ 0.04007501,  0.0539028 ,  0.02401057],
       [ 0.09998377,  0.08962039,  0.07233005],
       [ 0.08306152,  0.0926372 ,  0.06587007],
       [ 0.12716918,  0.10270973,  0.07002366],
       [ 0.06523076,  0.14404846,  0.01551149],
       [ 0.07320981,  0.01066726,  0.01494804],
       [ 0.11345544,  0.07224176,  0.07157639],
       [ 0.20894713,  0.14860273,  0.19899653],
       [ 0.08432908,  0.08875486,  0.050